<a href="https://colab.research.google.com/github/Currybroketherecord/AI-week7/blob/main/week7AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
dataset_orig = CompasDataset()

# Privileged and unprivileged groups
privileged_groups = [{'race': 1}]   # Caucasian
unprivileged_groups = [{'race': 0}] # African-American

# Bias Metrics Before Mitigation
metric_orig = BinaryLabelDatasetMetric(dataset_orig,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

print("Disparate Impact:", metric_orig.disparate_impact())
print("Statistical Parity Difference:", metric_orig.statistical_parity_difference())

# Preprocessing – Reweighing
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf = RW.fit_transform(dataset_orig)

# Train/test split
dataset_orig_train, dataset_orig_test = dataset_transf.split([0.7], shuffle=True)
X_train = dataset_orig_train.features
y_train = dataset_orig_train.labels.ravel()
X_test = dataset_orig_test.features
y_test = dataset_orig_test.labels.ravel()

# Model
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Create dataset with predictions
dataset_pred = dataset_orig_test.copy()
dataset_pred.labels = y_pred

# Fairness metrics
classified_metric = ClassificationMetric(dataset_orig_test,
                                         dataset_pred,
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

print("False Positive Rate difference:",
      classified_metric.false_positive_rate_difference())
print("Equal Opportunity Difference:",
      classified_metric.equal_opportunity_difference())

# Visualization
labels = ['False Positive Rate', 'False Negative Rate']
values = [
    classified_metric.false_positive_rate_difference(),
    classified_metric.false_negative_rate_difference()
]
sns.barplot(x=labels, y=values)
plt.title("Disparity Metrics (Unprivileged vs. Privileged)")
plt.ylabel("Difference")
plt.axhline(0, color='gray', linestyle='--')
plt.show()
